In [1]:
import ee
from datetime import date
import folium
import geemap
import ipyleaflet
import matplotlib.pyplot as plt
from geemap import cartoee
import numpy as np
import cartopy as ccrs
import geemap.colormaps as cm
import pandas as pd
import datetime
from datetime import datetime, timedelta
from collections import OrderedDict
from calendar import monthrange

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()
import sys
sys.path.append('C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\geeSEBAL_copy_edits\\etbrasil\\')
import geesebal
from geesebal import (tools,landsatcollection,masks,meteorology,endmembers, 
evapotranspiration,collection,timeseries,image,ET_Collection_mod)

### Objective 
1. Sensitivity Analysis of Albedo
    -Perturb albedo by 5%, 10% 
    -replace albedo by observed albedo
    -Observe the change in ET errors commensurately
- What to change in the algorithm: 
    - Keep the cold pixel and hot pixel locations the same
    - Only run the model for the dates when observed albedo is available(that will require a merger with observed data) 
- But the algorithm uses images, how do i use single pixel values
    - One option is to just just create an image of same pixel values and use it for computation 
    - Second option is to reduce the algorithm to only points 

In [3]:
def et_collection_SR(start_date,end_date,lon,lat,scale):
#     try:
#     ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('WRS_PATH', 'equals', path).filterMetadata('WRS_ROW', 'equals',row).filterMetadata('CLOUD_COVER', 'less_than', 10);
    geometry=ee.Geometry.Point([lon,lat])
    ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('CLOUD_COVER', 'less_than',80 )
#     .filterMetadata('CLOUD_COVER', 'less_than', 10);
    ls=ls.filterBounds(geometry)
    def applyScaleFactors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
    ls=ls.map(applyScaleFactors)
#    ls = ls.filter(ee.Filter.neq('system:index','LC08_038037_20210903'));
    ls_list=ls.aggregate_array('system:index').getInfo()
    print(ls_list)
    count = ls.size().getInfo()
    print("Number of scenes: ", count)
    n=0
    k=0
    lon_cold_pixel=[]
    lat_cold_pixel=[]
    ts_cold_scene=[]
    cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_temp,cold_pixel_sum=[],[],[],[],[]
    hot_pixel_lat,hot_pixel_lon,hot_pixel_ndvi,hot_pixel_temp,hot_pixel_sum=[],[],[],[],[]
    hot_pixel_Rn,hot_pixel_G=[],[]
    zenith_angle=[]
    #====== ITERATIVE PROCESS ======#
    #FOR EACH IMAGE ON THE LIST
    #ESTIMATE ET DAILY IMAGE
    while n < count:
        #GET IMAGE
#         for i in range(var_merged.shape[0]):
        image= ls.filterMetadata('system:index','equals',ls_list[n]).first()
        image.getInfo()
        image=ee.Image(image)
            # et=image.Image(image)
        NDVI_cold=5
        Ts_cold=20
        NDVI_hot=10
        Ts_hot=20
        index=image.get('system:index')
        cloud_cover=image.get('CLOUD_COVER')
        LANDSAT_ID=image.get('L1_LANDSAT_PRODUCT_ID').getInfo()
        print(LANDSAT_ID)
        landsat_version=image.get('SATELLITE').getInfo()
        sun_elevation=image.get("SUN_ELEVATION")
        print(sun_elevation.getInfo())
        time_start=image.get('system:time_start')
        date=ee.Date(time_start)
        year=ee.Number(date.get('year'))
        month=ee.Number(date.get('month'))
        day=ee.Number(date.get('day'))
        hour=ee.Number(date.get('hour'))
        minuts = ee.Number(date.get('minutes'))
        print(str(hour.getInfo())+str(minuts.getInfo()))
        crs = image.projection().crs()
        transform=ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))
        date_string=date.format('YYYY-MM-dd').getInfo()
        #ENDMEMBERS
        p_top_NDVI=ee.Number(NDVI_cold)
        p_coldest_Ts=ee.Number(Ts_cold)
        p_lowest_NDVI=ee.Number(NDVI_hot)
        p_hottest_Ts=ee.Number(Ts_hot)
        ls_trial=image.select([0,1,2,3,4,5,6,8,17], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","ST_B10","pixel_qa"])
#       ls.first_toa=ee.Image('LANDSAT/LC08/C01/T1/'+index.getInfo())
        print(ls_trial.bandNames().getInfo())

#col_rad = ee.Algorithms.Landsat.calibratedRadiance(ls.first_toa)
#col_rad = ls_trial.addBands(col_rad.select([9],["T_RAD"]))
#CLOUD REMOVAL
#ls_trial=ee.ImageCollection(col_rad).map(masks.f_cloudMaskL8_SR)
        ls_trial=masks.f_cloudMaskL8_SR(ls_trial)
#         print("Cloud masking Complete")
        print(ls_trial.bandNames().getInfo())

        #ALBEDO TASUMI ET AL. (2008) METHOD WITH KE ET AL. (2016) COEFFICIENTS
        # ls_trial=ls_trial.map(masks.f_albedoL8)
        ls_trial=masks.f_albedoL8(ls_trial)
        print(ls_trial.bandNames().getInfo())
#         print("Albedo calc done")

        #------ Meteorology
          #GEOMETRY
        geometryReducer=ls_trial.geometry().bounds().getInfo()
#         print("sun elevation check")

        geometry_download=geometryReducer['coordinates']

        # camada_clip=ls_trial.select('BRT').first()
#         camada_clip=ls_trial.select('BRT')
#         sun_elevation=ee.Number(90).subtract(ee.Number(azimuth_angle))
        print(sun_elevation.getInfo())
        #METEOROLOGY PARAMETERS
        col_meteorology= meteorology.get_meteorology(ls_trial,time_start);
        #AIR TEMPERATURE [C]
        T_air = col_meteorology.select('AirT_G');
        print(T_air.bandNames().getInfo())
        #WIND SPEED [M S-1]
        ux= col_meteorology.select('ux_G');
        #RELATIVE HUMIDITY [%]
        UR = col_meteorology.select('RH_G');

        #NET RADIATION 24H [W M-2]
        Rn24hobs = col_meteorology.select('Rn24h_G');

## print("Metorology ready")

        #------
        #------ Elevation
        #SRTM DATA ELEVATION
        SRTM_ELEVATION ='USGS/SRTMGL1_003'
        srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
        z_alt = srtm.select('elevation')
## print(z_alt) 
        ls_trial=tools.fexp_spec_ind(ls_trial)
        ls_trial=tools.LST_DEM_correction(ls_trial, z_alt, T_air, UR,sun_elevation,hour,minuts)
        print("It's a miracle")
## GET IMAGE
## COLD PIXEL
        d_cold_pixel=endmembers.fexp_cold_pixel(ls_trial, geometryReducer, p_top_NDVI, p_coldest_Ts)
        print(d_cold_pixel.getInfo())
## COLD PIXEL NUMBER
        n_Ts_cold = ee.Number(d_cold_pixel.get('temp').getInfo())
##INSTANTANEOUS OUTGOING LONG-WAVE RADIATION [WM-2]
        ls_trial=tools.fexp_radlong_up(ls_trial)
##INSTANTANEOUS INCOMING SHORT-WAVE RADIATION [WM-2]
        ls_trial=tools.fexp_radshort_down(ls_trial,z_alt,T_air,UR, sun_elevation)

## INSTANTANEOUS INCOMING LONGWAVE RADIATION [W M-2]
        ls_trial=tools.fexp_radlong_down(ls_trial, n_Ts_cold)
##INSTANTANEOUS NET RADIATON BALANCE [W M-2]

        ls_trial=tools.fexp_radbalance(ls_trial)

##SOIL HEAT FLUX (G) [W M-2]
        ls_trial=tools.fexp_soil_heat(ls_trial)
##HOT PIXEL
        d_hot_pixel=endmembers.fexp_hot_pixel(ls_trial, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
##SENSIBLE HEAT FLUX (H) [W M-2]
        ls_trial=tools.fexp_sensible_heat_flux(ls_trial, ux, UR,Rn24hobs,n_Ts_cold,
                                       d_hot_pixel, date_string,geometryReducer)

##DAILY EVAPOTRANSPIRATION (ET_24H) [MM DAY-1]
        ls_trial=evapotranspiration.fexp_et(ls_trial,Rn24hobs)
## Store all the values of the cold pixel 
        cold_pixel_lat.append(d_cold_pixel.get("y").getInfo())
        cold_pixel_lon.append(d_cold_pixel.get("x").getInfo())
        cold_pixel_temp.append(d_cold_pixel.get("temp").getInfo())
        cold_pixel_ndvi.append(d_cold_pixel.get("ndvi").getInfo())
        cold_pixel_sum.append(d_cold_pixel.get("sum").getInfo())
## Get info about hot pixl
        hot_pixel_lat.append(d_hot_pixel.get("y").getInfo())
        hot_pixel_lon.append(d_hot_pixel.get("x").getInfo())
        hot_pixel_temp.append(d_hot_pixel.get("temp").getInfo())
        hot_pixel_ndvi.append(d_hot_pixel.get("ndvi").getInfo())
        hot_pixel_sum.append(d_hot_pixel.get("sum").getInfo())
        hot_pixel_Rn.append(d_hot_pixel.get("Rn").getInfo())
        hot_pixel_G.append(d_hot_pixel.get("G").getInfo())
        zenith_angle.append(90-sun_elevation.getInfo())

        NAME_FINAL=LANDSAT_ID[:5]+LANDSAT_ID[10:17]+LANDSAT_ID[17:25]
        if k ==0:



              #AIR TEMPERATURE [C]
#             Rl_out=ls_trial.select(['Rl_up'],[NAME_FINAL])
#             Rl_in=ls_trial.select(['Rn'],[NAME_FINAL])
#             tao_sw_1=ls_trial.select(["T_LST_DEM"],[NAME_FINAL])
#             Rn=ls_trial.select(['Rn'],[NAME_FINAL])
#             G=ls_trial.select(['G'],[NAME_FINAL])
#             H=ls_trial.select(['H'],[NAME_FINAL])
#             LE_inst=ls_trial.select(['LE'],[NAME_FINAL])
#             Rn_daily=ls_trial.select(['ST_B10'],[NAME_FINAL])
#             ET=ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL])
            new_ls=ee.List([])
            met=ee.List([])
            new_ls=new_ls.add(ls_trial)
            met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))
#             collectionFromConstructor=ee.ImageCollection()
#             collectionFromConstructor = ee.ImageCollection(ls_trial)



        else:
#             Rl_out=Rl_out.addBands(ls_trial.select(['Rl_up'],[NAME_FINAL]))
#             Rl_in=Rl_in.addBands(ls_trial.select(['Rn'],[NAME_FINAL]))
#             tao_sw_1=tao_sw_1.addBands(ls_trial.select(["T_LST_DEM"],[NAME_FINAL]))
#             Rn=Rn.addBands(ls_trial.select(['Rn'],[NAME_FINAL]))
#             G=G.addBands(ls_trial.select(['G'],[NAME_FINAL]))
#             H=H.addBands(ls_trial.select(['H'],[NAME_FINAL]))
#             LE_inst=LE_inst.addBands(ls_trial.select(['LE'],[NAME_FINAL]))
#             Rn_daily=Rn_daily.addBands(ls_trial.select(['ST_B10'],[NAME_FINAL]))
#             ET=ET.addBands(ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL]))
#             collectionFromConstructor = collectionFromConstructor.merge(ee.ImageCollection(ls_trial.select(['ET_24h',"ST_B10","H","G"],[NAME_FINAL])))
#             print('collectionFromConstructor: ', collectionFromConstructor);
            new_ls=new_ls.add(ls_trial)
            met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))





        k=k+1
        print(k)
        n=n+1
        print(n)
        ## Convert to dataframe 
        et_collection=ee.ImageCollection(new_ls)
        met_collection=ee.ImageCollection(met)

        ##Stuff t

        region = et_collection.getRegion(geometry, int(scale)).getInfo()
        era5_met=met_collection.getRegion(geometry, int(scale)).getInfo()
#     except:
#         return pd.DataFrame()
#       print('Value = %0.2f'%region[1][4])
# stuff the values in a dataframe for convenience      
    df = pd.DataFrame.from_records(region[1:len(region)])
    df_met = pd.DataFrame.from_records(era5_met[1:len(era5_met)])

    if df.shape == (0,0):
        return pd.DataFrame()
    else:

        # use the first list item as column names
        df.columns = region[0]
        df_met.columns=era5_met[0]
        df_met=df_met.drop(["time"],axis=1)
        print(df_met)
        df=pd.concat([df,df_met],axis=1)
#         cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_temp,cold_pixel_sum
        df["cold_pixel_lat"]=cold_pixel_lat
        df["cold_pixel_lon"]=cold_pixel_lon
        df["cold_pixel_ndvi"]=cold_pixel_ndvi
        df["cold_pixel_sum"]=cold_pixel_sum
        df["cold_pixel_temp"]=cold_pixel_temp

## Hot pixel
        df["hot_pixel_sum"]=hot_pixel_sum
        df["hot_pixel_lat"]=hot_pixel_lat
        df["hot_pixel_lon"]=hot_pixel_lon
        df["hot_pixel_ndvi"]=hot_pixel_ndvi
        df["hot_pixel_Rn"]=hot_pixel_Rn
        df["hot_pixel_G"]=hot_pixel_G
        df["hot_pixel_temp"]=hot_pixel_temp
      #drop id column(little value / overhead)
      #df.drop('id',axis=1,inplace=True)

      # divide the time field by 1000 as in milliseconds
      # while datetime takes seconds to convert unix time
      # to dates
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')
        
      # add the product name and latitude, longitude as a column
      # just to make sense of the returned data after the fact
#      df['product'] = pd.Series(product, index = df.index)

      # return output
        return df
        
#     return(albedo,tao_sw)
#     return(new_ls)
# re
# et=et_collection_SR("2021-01-01","2021-03-20",-121.4993,38.0992,30)
# rs=et_collection_SR("2021-01-01","2021-12-31",44,33)
# alb_gee=alb_gee.reproject(crs="EPSG:4326",scale=30)
# tao_gee=tao_gee.reproject(crs="EPSG:4326",scale=30)
# srtm=srtm.reproject(crs="EPSG:4326",scale=30)


In [4]:
var_merged=pd.read_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Test_files\\US-Var.csv",parse_dates=["date"])

In [5]:
# var_merged[var_merged["Albedo"].notna()]["NIR"]
var_merged["id"]
var_merged.columns

Index(['Unnamed: 0', 'id', 'longitude', 'latitude', 'date', 'UB', 'B', 'GR',
       'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI',
       'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude.1',
       'latitude.1', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI',
       'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw',
       'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down',
       'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1',
       'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF', 'id.1',
       'longitude.2', 'latitude.2', 'Rn24h_G', 'AirT_G', 'RH_G', 'ux_G',
       'SW_Down', 'cold_pixel_lat', 'cold_pixel_lon', 'cold_pixel_ndvi',
       'cold_pixel_sum', 'cold_pixel_temp', 'hot_pixel_sum', 'hot_pixel_lat',
       'hot_pixel_lon', 'hot_pixel_ndvi', 'hot_pixel_Rn', 'hot_pixel_G',
       'hot_pixel_temp'],
      dtype='object')

In [6]:
## Check how many images are there between 2015 and 2022
geometry=ee.Geometry.Point([ -120.9509216,38.41335394])
ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate("2016-01-01","2022-12-31").filterMetadata('CLOUD_COVER', 'less_than',100 )
ls=ls.filterBounds(geometry)

In [7]:
ls.size().getInfo()
image= ls.filterMetadata('system:index','equals',var_merged["id"].iloc[0]).first()
if image.getInfo():
    print("true")
ee.Imageimage.not()

SyntaxError: invalid syntax (4057296323.py, line 5)

### Modifyin the ET algorithm to keep cold and hot pixels the same
- Pass arguments in the function to generate dataframes with different sensitivity
    - For now we can only vary one variable and a list passed for the variation 
        - for 5%,10% and 15% give 1.05,1.1 and 1.15 and for negatives give 0.95,0.9 and 0.85

In [8]:
def et_collection_SR(start_date,end_date,lon,lat,scale,sen_var,magnitude):
    geometry=ee.Geometry.Point([lon,lat])
    ls=ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate(start_date,end_date).filterMetadata('CLOUD_COVER', 'less_than',80 )
    ls=ls.filterBounds(geometry)
    def applyScaleFactors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)
    ls=ls.map(applyScaleFactors)
    zenith_angle=[]
    image_id=[]
    new_ls=ee.List([])
    met=ee.List([])
    for i in range(var_merged.shape[0]):
        image= ls.filterMetadata('system:index','equals',var_merged["id"].iloc[i]).first()
        if image.getInfo():
            for j in range(len(magnitude)):
                print(var_merged["id"].iloc[i])
                print(i)
                image=ee.Image(image)
                    # et=image.Image(image)
                index=image.get('system:index')
                cloud_cover=image.get('CLOUD_COVER')
                LANDSAT_ID=image.get('L1_LANDSAT_PRODUCT_ID').getInfo()
                image_id.append(LANDSAT_ID)
                print(LANDSAT_ID)
#                 landsat_version=image.get('SATELLITE').getInfo()
                sun_elevation=image.get("SUN_ELEVATION")
                print(sun_elevation.getInfo())
                time_start=image.get('system:time_start')
                date=ee.Date(time_start)
                year=ee.Number(date.get('year'))
                month=ee.Number(date.get('month'))
                day=ee.Number(date.get('day'))
                hour=ee.Number(date.get('hour'))
                minuts = ee.Number(date.get('minutes'))
                print(str(hour.getInfo())+str(minuts.getInfo()))
                crs = image.projection().crs()
                transform=ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))
                date_string=date.format('YYYY-MM-dd').getInfo()
                #ENDMEMBERS
#                 p_top_NDVI=ee.Number(NDVI_cold)
#                 p_coldest_Ts=ee.Number(Ts_cold)
#                 p_lowest_NDVI=ee.Number(NDVI_hot)
#                 p_hottest_Ts=ee.Number(Ts_hot)
                ls_trial=image.select([0,1,2,3,4,5,6,8,17], ["UB","B","GR","R","NIR","SWIR_1","SWIR_2","ST_B10","pixel_qa"])
            #       ls.first_toa=ee.Image('LANDSAT/LC08/C01/T1/'+index.getInfo())
#                 print(ls_trial.bandNames().getInfo())

            #col_rad = ee.Algorithms.Landsat.calibratedRadiance(ls.first_toa)
            #col_rad = ls_trial.addBands(col_rad.select([9],["T_RAD"]))
            #CLOUD REMOVAL
            #ls_trial=ee.ImageCollection(col_rad).map(masks.f_cloudMaskL8_SR)
                ls_trial=masks.f_cloudMaskL8_SR(ls_trial)
            #         print("Cloud masking Complete")
#                 print(ls_trial.bandNames().getInfo())

                #ALBEDO TASUMI ET AL. (2008) METHOD WITH KE ET AL. (2016) COEFFICIENTS
                # ls_trial=ls_trial.map(masks.f_albedoL8)
                ls_trial=masks.f_albedoL8(ls_trial)

                print(ls_trial.bandNames().getInfo())
            #         print("Albedo calc done")

                #------ Meteorology
                  #GEOMETRY
                geometryReducer=ls_trial.geometry().bounds().getInfo()
            #         print("sun elevation check")

                geometry_download=geometryReducer['coordinates']

                # camada_clip=ls_trial.select('BRT').first()
            #         camada_clip=ls_trial.select('BRT')
            #         sun_elevation=ee.Number(90).subtract(ee.Number(azimuth_angle))
                print(sun_elevation.getInfo())
                #METEOROLOGY PARAMETERS
                col_meteorology= meteorology.get_meteorology(ls_trial,time_start);
                #AIR TEMPERATURE [C]
                T_air = col_meteorology.select('AirT_G');
                print(T_air.bandNames().getInfo())
                #WIND SPEED [M S-1]
                ux= col_meteorology.select('ux_G');
                #RELATIVE HUMIDITY [%]
                UR = col_meteorology.select('RH_G');

                #NET RADIATION 24H [W M-2]
                Rn24hobs = col_meteorology.select('Rn24h_G');

            ## print("Metorology ready")

                #------
                #------ Elevation
                #SRTM DATA ELEVATION
                SRTM_ELEVATION ='USGS/SRTMGL1_003'
                srtm = ee.Image(SRTM_ELEVATION).clip(geometryReducer);
                z_alt = srtm.select('elevation')
            ## print(z_alt) 
                ls_trial=tools.fexp_spec_ind(ls_trial)
                ls_trial=tools.LST_DEM_correction(ls_trial, z_alt, T_air, UR,sun_elevation,hour,minuts)
                print("It's a miracle")
            ## GET IMAGE
            ## COLD PIXEL
            #         d_cold_pixel=endmembers.fexp_cold_pixel(ls_trial, geometryReducer, p_top_NDVI, p_coldest_Ts)
            #         print(d_cold_pixel.getInfo())
            ## COLD PIXEL NUMBER
                n_Ts_cold = ee.Number(var_merged['cold_pixel_temp'].iloc[i])

            ##INSTANTANEOUS OUTGOING LONG-WAVE RADIATION [WM-2]
                ls_trial=tools.fexp_radlong_up(ls_trial)
                if sen_var=="LW_OUT":
                    LW_OUT= ls_trial.expression(str(magnitude[j])+'*Rl_up', {'Rl_up': ls_trial.select('Rl_up')}).rename("Rl_up");
                    ls_trial=ls_trial.addBands(LW_OUT,overwrite=True).toFloat()
            ##INSTANTANEOUS INCOMING SHORT-WAVE RADIATION [WM-2]
                ls_trial=tools.fexp_radshort_down(ls_trial,z_alt,T_air,UR, sun_elevation)

            ## INSTANTANEOUS INCOMING LONGWAVE RADIATION [W M-2]
                ls_trial=tools.fexp_radlong_down(ls_trial, n_Ts_cold)
            ##INSTANTANEOUS NET RADIATON BALANCE [W M-2]

                ls_trial=tools.fexp_radbalance(ls_trial)

            ##SOIL HEAT FLUX (G) [W M-2]
                ls_trial=tools.fexp_soil_heat(ls_trial)
            ##HOT PIXEL
            #         d_hot_pixel=endmembers.fexp_hot_pixel(ls_trial, geometryReducer,p_lowest_NDVI, p_hottest_Ts)
                d_hot_pixel = ee.Dictionary({
                        'temp': ee.Number(var_merged['hot_pixel_temp'].iloc[i].tolist()),
                        'x': ee.Number(var_merged['hot_pixel_lon'].iloc[i].tolist()),
                        'y': ee.Number(var_merged['hot_pixel_lat'].iloc[i].tolist()),
                        'Rn': ee.Number(var_merged['hot_pixel_Rn'].iloc[i].tolist()),
                        'G': ee.Number(var_merged['hot_pixel_G'].iloc[i].tolist()),
                        'ndvi': ee.Number(var_merged['hot_pixel_ndvi'].iloc[i].tolist()),
                        'sum': ee.Number(var_merged['hot_pixel_sum'].iloc[i].tolist())})

            ##SENSIBLE HEAT FLUX (H) [W M-2]
                ls_trial=tools.fexp_sensible_heat_flux(ls_trial, ux, UR,Rn24hobs,n_Ts_cold,
                                               d_hot_pixel, date_string,geometryReducer)

            ##DAILY EVAPOTRANSPIRATION (ET_24H) [MM DAY-1]
                ls_trial=evapotranspiration.fexp_et(ls_trial,Rn24hobs)
                zenith_angle.append(90-sun_elevation.getInfo())
                NAME_FINAL=LANDSAT_ID[:5]+LANDSAT_ID[10:17]+LANDSAT_ID[17:25]
#                 if i==0 and j==0:
#                     new_ls=ee.List([])
#                     met=ee.List([])
                new_ls=new_ls.add(ls_trial)
                met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))
                print("Added First")
                print(ls_trial.bandNames().getInfo())
#                 else:
#                     new_ls=new_ls.add(ls_trial)
#                     met=met.add(col_meteorology.select("Rn24h_G","AirT_G","RH_G","ux_G","SW_Down"))
#                     print("second done too")
#                     print(ls_trial.bandNames().getInfo())


        else: 
            continue
   
    et_collection=ee.ImageCollection(new_ls)
    met_collection=ee.ImageCollection(met)
    ##Stuff t
    region = et_collection.getRegion(geometry, int(scale)).getInfo()
    era5_met=met_collection.getRegion(geometry, int(scale)).getInfo()
    #     except:
    #         return pd.DataFrame()
    #       print('Value = %0.2f'%region[1][4])
    # stuff the values in a dataframe for convenience      
    df = pd.DataFrame.from_records(region[1:len(region)])
    df_met = pd.DataFrame.from_records(era5_met[1:len(era5_met)])

    if df.shape == (0,0):
        return pd.DataFrame()
    else:

        # use the first list item as column names
        df.columns = region[0]
        df_met.columns=era5_met[0]
        df_met=df_met.drop(["time"],axis=1)
        print(df_met)
        df=pd.concat([df,df_met],axis=1)
        df["image_id"]=image_id
      # divide the time field by 1000 as in milliseconds
      # while datetime takes seconds to convert unix time
      # to dates
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')
      # return output
        return df

# temp1=et_collection_SR("2015-07-01","2015-07-15",-120.9509216,38.41335394,30,"LW_OUT",[0.85,0.9,1,1.05,1.15])

In [10]:

def split_years(start_date, end_date):
    """
    Function to split query period to bypass computation timed out (300s query limit of gee)
    Takes input as start and end years
    Gives output as list of start and end periods for each year
    """
  
    start, end = [datetime.strptime(_, "%Y-%m-%d") for _ in [start_date, end_date]]
    start = list(OrderedDict(((start + timedelta(_)).strftime(r"%Y-%m-01"), None) for _ in range((end - start).days)).keys())
    end = [item[:-2]+str(monthrange(int(item[:4]),int(item[5:7]))[1]) for item in start] 
#    print(end)
    return start,end
a,b=split_years("2020-01-01","2020-02-01")
# list(a)
a

['2020-01-01']

In [11]:
 def call_et_func(lon,lat,start_date,end_date,scale,var_name,magnitude):
        start,end=split_years(start_date,end_date)
        print(len(start))
        concat=pd.DataFrame()
        for i in range(len(start)):
                try:
                    df_sub=et_collection_SR(start[i],end[i],lon,lat,scale,var_name,magnitude)
#                     concat = pd.concat([concat, df_sub], axis = 0) 
                    df_sub.to_csv("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Sensitivity\\LW_OUT\\Intermittent_Files\\"+str(end[i])+".csv")
                except NameError:
                    print("Error: check input parameters")
        return concat
var=call_et_func(-120.9509216,38.41335394,"2022-08-01","2022-12-31",30,"LW_OUT",[0.85,0.9,1,1.05,1.15])

5
LC08_043033_20220808
442
LC08_L1TP_043033_20220808_20220818_02_T1
60.29896371
1839
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
60.29896371
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_043033_20220808
442
LC08_L1TP_043033_20220808_20220818_02_T1
60.29896371
1839
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
60.29896371
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAV

LC08_L1TP_043034_20220808_20220818_02_T1
61.08851135
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
61.08851135
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_043034_20220808
444
LC08_L1TP_043034_20220808_20220818_02_T1
61.08851135
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
61.08851135
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 

LC08_L1TP_044033_20220815_20220823_02_T1
58.72199076
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
58.72199076
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
    id   longitude   latitude     Rn24h_G     AirT_G       RH_G      ux_G  \
0    0 -120.950922  38.413354  149.998057  25.337653  39.411269  1.398085   
1    1 -120.950922  38.413354  149.998057  25.337653  39.411269  1.398085   
2    2 -120.950922  38.413354  149.998057  25.337653  39.411269  1.398085   
3    3 -120.950922  38.413354  

LC08_L1TP_043033_20220925_20221004_02_T1
46.73836003
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
46.73836003
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_043033_20220925
448
LC08_L1TP_043033_20220925_20221004_02_T1
46.73836003
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
46.73836003
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 

LC08_044033_20220916
451
LC08_L1TP_044033_20220916_20220922_02_T1
49.69071259
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
49.69071259
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_044033_20220916
451
LC08_L1TP_044033_20220916_20220922_02_T1
49.69071259
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
49.69071259
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI'

...

38.94732963
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
38.94732963
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_044033_20221018
457
LC08_L1TP_044033_20221018_20221031_02_T1
38.94732963
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
38.94732963
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_ne

27.59784726
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
27.59784726
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_043033_20221128
459
LC08_L1TP_043033_20221128_20221206_02_T1
27.59784726
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
27.59784726
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_ne

28.87275742
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
28.87275742
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_043034_20221128
461
LC08_L1TP_043034_20221128_20221206_02_T1
28.87275742
1840
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
28.87275742
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_ne

LC08_043033_20221214
464
LC08_L1TP_043033_20221214_20221223_02_T1
25.40156938
1839
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
25.40156938
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_043033_20221214
464
LC08_L1TP_043033_20221214_20221223_02_T1
25.40156938
1839
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
25.40156938
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI'

LC08_L1TP_044033_20221205_20221212_02_T1
26.43350695
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
26.43350695
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 'e_NB', 'longitude', 'latitude', 'NDVI_neg', 'pos_NDVI', 'int', 'sd_ndvi', 'NDWI', 'LST_neg', 'LST_NW', 'T_LST_DEM', 'LST_neg_1', 'LST_NW_1', 'Tao_sw', 'EA', 'W_ATM', 'Rl_up', 'Rs_down', 'Tao_sw_1', 'ES', 'EA_1', 'Rl_down', 'Rn', 'G', 'zom', 'u_fr', 'H', 'rah', 'dT', 'rah_first', 'zom_1', 'u_fr_1', 'ufric_star', 'ET_inst', 'ET_24h', 'LE', 'EF']
LC08_044033_20221205
466
LC08_L1TP_044033_20221205_20221212_02_T1
26.43350695
1846
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA']
26.43350695
['AirT_G']
It's a miracle
Added First
['UB', 'B', 'GR', 'R', 'NIR', 'SWIR_1', 'SWIR_2', 'ST_B10', 'pixel_qa', 'ALFA', 'NDVI', 'EVI', 'SAVI', 'T_LST', 'LAI', 'e_0', 

In [ ]:
var

In [ ]:
var_merged

In [ ]:
ee.Number(var_merged['hot_pixel_temp'].iloc[0].tolist()).getInfo()
# var_merged['hot_pixel_temp'].iloc[0].tolist()

In [ ]:
d_hot_pixel = ee.Dictionary({
                'temp': ee.Number(var_merged['hot_pixel_temp'].iloc[0].tolist()),
                'x': ee.Number(var_merged['hot_pixel_lon'].iloc[0].tolist()),
                'y': ee.Number(var_merged['hot_pixel_lat'].iloc[0].tolist()),
                'Rn': ee.Number(var_merged['hot_pixel_Rn'].iloc[0].tolist()),
                'G': ee.Number(var_merged['hot_pixel_G'].iloc[0].tolist()),
                'ndvi': ee.Number(var_merged['hot_pixel_ndvi'].iloc[0].tolist()),
                'sum': ee.Number(var_merged['hot_pixel_sum'].iloc[0].tolist())})

In [ ]:
var_merged.columns

In [ ]:
var_merged[["id","AirT_G","H","LE","ALFA","Rl_up"]].head()

In [ ]:
temp1[["image_id","AirT_G","H","LE","ALFA","Rl_up"]]

In [ ]:
temp1.iloc[1:12:2]
magnitude=[0.85,0.9,1,1.1,1.15]
trial=[temp1.iloc[i:temp1.shape[0]:len(magnitude)] for i in range(len(magnitude))]
trial[1]

In [ ]:
trial[5]